# Cloth Wraping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
cd /content/

/content


In [ ]:
!pwd

/content


In [ ]:
%rm -r inputs/*
%rm -r inputs_cloth/*

In [ ]:
! mkdir inputs
! mkdir inputs_cloth

mkdir: cannot create directory ‘inputs’: File exists
mkdir: cannot create directory ‘inputs_cloth’: File exists


In [ ]:
from google.colab import files
%cd inputs
print('Upload the model image : ')
uploaded = files.upload()
%cd ..

/content/inputs
Upload the model image : 


Saving 00094_00.jpg to 00094_00.jpg
/content


In [ ]:
from google.colab import files
%cd inputs_cloth
print('upload the cloth image : ')
uploaded = files.upload()
%cd ..

/content/inputs_cloth
upload the cloth image : 


Saving cloth1.jpg to cloth1.jpg
/content


In [ ]:
import os
model_image = os.listdir("/content/inputs")
cloth_image = os.listdir("/content/inputs_cloth")

model_image = model_image[0]
cloth_image = cloth_image[0]

In [ ]:
print(model_image)
print(cloth_image)

00094_00.jpg
cloth1.jpg


In [ ]:
ls -l

total 28
drwx------ 5 root root 4096 Aug 19 14:37 drive/
drwxr-xr-x 4 root root 4096 Aug 19 14:40 final_output/
drwxr-xr-x 2 root root 4096 Aug 19 14:44 inputs/
drwxr-xr-x 2 root root 4096 Aug 19 14:44 inputs_cloth/
drwxr-xr-x 2 root root 4096 Aug 19 14:38 inputs_cloth_mask/
drwxr-xr-x 2 root root 4096 Aug 19 14:38 inputs_cloth_mask_modified/
drwxr-xr-x 1 root root 4096 Aug 15 13:44 sample_data/


In [ ]:
import glob 

paths = glob.glob('./inputs/*')
paths.extend(glob.glob('./inputs_cloth/*'))
for i in paths:
  if '.jpg' not in i:
    os.remove(i)

In [ ]:
import cv2 
import os 

for i in os.listdir('./inputs/'):
    print(i)
    im = cv2.imread('./inputs/'+i)
    print(im.shape)
    im = cv2.resize(im,(768,1024))
    print(im.shape)
    cv2.imwrite('./inputs/'+i.split('.')[0]+'.jpg',im)

00094_00.jpg
(1024, 768, 3)
(1024, 768, 3)


In [ ]:
import cv2 
import os 

for i in os.listdir('./inputs_cloth/'):
    print(i)
    im = cv2.imread('./inputs_cloth/'+i)
    print(im.shape)
    im = cv2.resize(im,(768,1024))
    print(im.shape)
    cv2.imwrite('./inputs_cloth/'+i.split('.')[0]+'.jpg',im)

cloth1.jpg
(869, 670, 3)
(1024, 768, 3)


## Cloth Segmentation

In [ ]:
%cd /content/

/content


In [ ]:
! rm -r inputs_cloth_mask

In [ ]:
rm -r /content/inputs_cloth/.ipynb_checkpoints

rm: cannot remove '/content/inputs_cloth/.ipynb_checkpoints': No such file or directory


In [ ]:
! mkdir inputs_cloth_mask

In [ ]:
%cd /content/drive/MyDrive/hr-viton/pipeline/HumanPose/clothsegment

/content/drive/MyDrive/hr-viton/pipeline/HumanPose/clothsegment


In [ ]:
! python3 infer.py

----checkpoints loaded from path: trained_checkpoint/cloth_segm_u2net_latest.pth----
  0% 0/1 [00:00<?, ?it/s]['cloth1.jpg']
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3722: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
100% 1/1 [00:17<00:00, 17.15s/it]


In [ ]:
%cd /content/

/content


In [ ]:
%rm -r inputs_cloth_mask_modified

In [ ]:
from PIL import Image, ImageChops
import numpy as np

In [ ]:
! mkdir inputs_cloth_mask_modified
images_list = os.listdir('./inputs_cloth_mask')
for image in images_list:
  image_path = './inputs_cloth_mask/' + image
  image_save_path = './inputs_cloth_mask_modified/' + image.split('.')[0] + '.jpg'
  img = Image.open(image_path)
  img = img.convert("RGB")
  width, height = img.size
  print(width, height)
  for i in range(0, width):  					  
    for j in range(0, height):  				  
      data = img.getpixel((i, j))  				  
      if (data[0] >= 64):
        img.putpixel((i, j), (255, 255, 255))
  img = img.convert('L')
  img.save(image_save_path)  		 

768 1024


## Wraping Model

In [ ]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!nvidia-smi

Fri Aug 19 14:45:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install torchgeometry

In [ ]:
%cd /content/drive/MyDrive/hr-viton/pipeline/HumanPose/HR-VITON

/content/drive/MyDrive/hr-viton/pipeline/HumanPose/HR-VITON


In [ ]:
%rm -f ./my_data/test/cloth/*
%rm -f ./my_data/test/cloth-mask/*
%rm -f ./my_data/test/image/*
%rm -f ./my_data/test/image-densepose/*
%rm -f ./my_data/test/image-parse-agnostic-v3.2/*
%rm -f ./my_data/test/image-parse-v3/*
%rm -f ./my_data/test/openpose_img/*
%rm -f ./my_data/test/openpose_json/*
%rm ./my_data/test_pairs.txt

In [ ]:
with open('./my_data/test_pairs.txt', 'w') as f:
    f.write(model_image+" "+cloth_image)

In [ ]:
import shutil

In [ ]:
from IPython.core.display import JSON

%cp -r /content/inputs_cloth/* ./my_data/test/cloth
%cp -r /content/inputs_cloth_mask_modified/* ./my_data/test/cloth-mask
%cp -r /content/inputs/* ./my_data/test/image

src = os.path.join('data/test/image-densepose', model_image)
dst = os.path.join('my_data/test/image-densepose', model_image)
shutil.copyfile(src, dst)

src = os.path.join('data/test/image-parse-v3', model_image.replace("jpg", "png"))
dst = os.path.join('my_data/test/image-parse-v3', model_image.replace("jpg", "png"))
shutil.copyfile(src, dst)

src = os.path.join('data/test/image-parse-agnostic-v3.2', model_image.replace("jpg", "png"))
dst = os.path.join('my_data/test/image-parse-agnostic-v3.2', model_image.replace("jpg", "png"))
shutil.copyfile(src, dst)

src = os.path.join('data/test/openpose_img', (model_image[0:8]+"_rendered"+model_image[8:]).replace("jpg","png"))
dst = os.path.join('my_data/test/openpose_img', (model_image[0:8]+"_rendered"+model_image[8:]).replace("jpg","png"))
shutil.copyfile(src, dst)

src = os.path.join('data/test/openpose_json', (model_image[0:8]+"_keypoints"+model_image[8:]).replace("jpg","json"))
dst = os.path.join('my_data/test/openpose_json', (model_image[0:8]+"_keypoints"+model_image[8:]).replace("jpg","json"))
shutil.copyfile(src, dst)

'my_data/test/openpose_json/00094_00_keypoints.json'

In [ ]:
!pip install opencv-python torchgeometry Pillow tqdm tensorboardX scikit-image scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 test_generator.py --occlusion --test_name test15 --tocg_checkpoint checkpoints/mtviton.pth --gpu_ids 0 --gen_checkpoint checkpoints/gen.pth --datasetting unpaired --dataroot my_data/ --data_list test_pairs.txt

Namespace(batch_size=1, checkpoint_dir='checkpoints', clothmask_composition='warp_grad', data_list='test_pairs.txt', datamode='test', dataroot='my_data/', datasetting='unpaired', fine_height=1024, fine_width=768, fp16=False, gen_checkpoint='checkpoints/gen.pth', gen_semantic_nc=7, gpu_ids='0', init_type='xavier', init_variance=0.02, ngf=64, norm_G='spectralaliasinstance', num_upsampling_layers='most', occlusion=True, out_layer='relu', output_dir=None, output_nc=13, semantic_nc=13, shuffle=False, tensorboard_count=100, tensorboard_dir='tensorboard', test_name='test15', tocg_checkpoint='checkpoints/mtviton.pth', upsample='bilinear', warp_feature='T1', workers=4)
Start to test %s!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation

In [ ]:
%cd /content/

/content


In [ ]:
%rm -r final_output/*

In [ ]:
!mkdir final_output

mkdir: cannot create directory ‘final_output’: File exists


In [ ]:
%cp -r /content/drive/MyDrive/hr-viton/pipeline/HumanPose/HR-VITON/output/test15/test/unpaired/generator/* final_output/

In [ ]:
#pip freeze > requirements.txt